# Grafos

## O que é um Grafo?

Pensem em um grafo como um **mapa de conexões**. Ajuda a representar visualmente coisas que estão interligadas.

Composição:

  * **Vértices (ou Nós):** São os "pontos" ou as "coisas" que queremos conectar.
  * **Arestas:** São as "linhas" ou as "conexões" que ligam um vértice a outro.

&nbsp;

<img src="https://algoritmosempython.com.br/images/algoritmos-python/algoritmos-grafos/ModelagemGrafosExemplo.png" width=400\>

&nbsp;

### Exemplos:

  * **Rede social (Instagram/Facebook):**

      * **Vértices:** Cada pessoa na rede é um vértice.
      * **Arestas:** A relação de "amizade" ou "seguir" é uma aresta.

  * **Mapa de vôos (Google Flights):**

      * **Vértices:** Cada cidade/aeroporto é um vértice.
      * **Arestas:** Uma rota de voo direta entre duas cidades é uma aresta.

## Tipos de conexões (Arestas)

1.  **Conexões com sentido único (Grafos direcionados)**

      * A conexão só vale em uma direção.
      * **Exemplo prático:** No Instagram, você pode seguir uma celebridade (aresta `Você -> Celebridade`), mas ela não precisa te seguir de volta. A conexão tem um sentido.

    <img src="https://www.ime.usp.br/~pf/algoritmos_para_grafos/aulas/figs/Sedgewick-Wayne/TinyNetworkOnly.png" width=400\>

&nbsp;

2.  **Conexões de mão dupla (Grafos não direcionados)**

      * A conexão é mútua. Se A está conectado a B, B também está conectado a A.
      * **Exemplo prático:** No Facebook, a amizade é de mão dupla. Se você é amigo de alguém, essa pessoa também é sua amiga. A conexão não tem uma "seta".

    <img src="https://algoritmosempython.com.br/images/algoritmos-python/algoritmos-grafos/ModelagemGrafosExemplo.png" width=400\>
    
&nbsp;

3.  **Conexões com valor (Grafos ponderados)**

      * A conexão tem um "custo" ou "peso" associado a ela.
      * **Exemplo Prático:** Em um mapa de voos, o **preço** da passagem entre duas cidades é o peso da aresta. Em um aplicativo como o Waze, a **distância em km** ou o **tempo de viagem** entre dois pontos são os pesos.

    <img src="https://upload.wikimedia.org/wikipedia/commons/thumb/a/a8/Prim_Algorithm_0.svg/250px-Prim_Algorithm_0.svg.png" width=400 style="background-color:white;">

### Como descrevemos um grafo para o computador?

#### 1\. Matriz de adjacência (A planilha de conexões)

Como uma planilha onde as linhas e as colunas são todos os vértices do seu grafo (ex: todas as cidades do mapa).

  * Para saber se existe uma conexão direta da **cidade A** para a **cidade B**, você olha na **linha A, coluna B**.
  * Se o valor for `1` (ou qualquer outro número diferente de zero), a conexão existe. Se for `0`, não existe uma conexão direta.

<img src="https://s3-sa-east-1.amazonaws.com/lcpi/1f5e0fd3-572c-479a-8a53-74d8212c5e5c.jpg" width=500\>

  * **Quando usar?** É ótima para grafos "densos", onde quase todo mundo está conectado com todo mundo.
  * **Desvantagem:** Ocupa muita memória para grafos "esparsos" (com poucas conexões, como uma rede social).

&nbsp;

A imagem acima representa as conexões de mão dupla (grafos não direcionados).

A matriz de adjacência também pode representar os grafos direcionados (conexões de mão única), onde a flecha indica indica qual o vértice de origem e qual o vértice de destino (a flecha "sai" da linha e "entra" na coluna.)

<img src="https://algoritmosempython.com.br/images/algoritmos-python/algoritmos-grafos/GrafoMatrizAdjacencia.png" width=400>

O código abaixo demonstra o exemplo de implementação de um grafo com a Matriz de Adjacências

In [ ]:
class GrafoMA:
    def __init__(self):
        # Dicionário para mapear rótulos (strings/ints) de nós para seus índices internos
        self.mapeamento_indices = {}
        # Lista para armazenar os rótulos dos nós na ordem dos índices
        self.rotulos_nos = []
        # A matriz de adjacências será inicializada vazia e expandida conforme necessário
        self.adj = []
        self.num_nos = 0 # Contador para o número atual de nós no grafo

    def __repr__(self):
        return self.__str__()
    
    def __str__(self):
        # Cria uma representação da matriz com os rótulos dos nós
        s = "  " + " ".join([str(rotulo) for rotulo in self.rotulos_nos]) + "\n"
        for i, linha in enumerate(self.adj):
            s += f"{self.rotulos_nos[i]} " + " ".join([str(peso) for peso in linha]) + "\n"
        return s.strip() # Remove a última linha vazia se houver

    def __obter_ou_criar_indice(self, no_label):
        """
        Retorna o índice de um nó existente ou cria um novo se não existir.
        Expande a matriz de adjacências se um novo nó for adicionado.
        """
        if no_label not in self.mapeamento_indices:
            # Atribui um novo índice ao rótulo do nó
            novo_indice = self.num_nos
            self.mapeamento_indices[no_label] = novo_indice
            self.rotulos_nos.append(no_label)
            self.num_nos += 1

            # Expande a matriz de adjacências
            # Adiciona uma nova coluna (0s) a todas as linhas existentes
            for linha_existente in self.adj:
                linha_existente.append(0)
            # Adiciona uma nova linha (0s) com o novo tamanho total
            self.adj.append([0] * self.num_nos)
            
            # Garante que as linhas existentes tenham o tamanho correto após adicionar a nova coluna
            for i in range(self.num_nos - 1): # Percorre todas as linhas existentes, exceto a recém-adicionada
                if len(self.adj[i]) < self.num_nos:
                    self.adj[i].extend([0] * (self.num_nos - len(self.adj[i])))

        return self.mapeamento_indices[no_label]

    def adicionar_vertice(self, no_label):
        self.__obter_ou_criar_indice(no_label)

    def adicionar_aresta(self, no1_label, no2_label, peso=1, bidirecional=True):
        # Obtém ou cria os índices internos para os rótulos dos nós
        idx1 = self.__obter_ou_criar_indice(no1_label)
        idx2 = self.__obter_ou_criar_indice(no2_label)
        
        # Garante que a matriz tenha o tamanho correto para os índices antes de acessar
        # (Isso é redundante se __obter_ou_criar_indice for chamado corretamente e expandir a matriz)
        # Mas serve como uma verificação extra robusta
        max_idx = max(idx1, idx2)
        if max_idx >= len(self.adj):
            # Isso não deveria acontecer se __obter_ou_criar_indice funciona corretamente
            # mas é um fallback de segurança
            self.adj.extend([[0] * self.num_nos for _ in range(max_idx - len(self.adj) + 1)])
            for i in range(len(self.adj)):
                if len(self.adj[i]) < self.num_nos:
                    self.adj[i].extend([0] * (self.num_nos - len(self.adj[i])))

        self.adj[idx1][idx2] = peso
        
        if bidirecional:
            self.adj[idx2][idx1] = peso
            
    def obter_vizinhos(self, no_label):
        # Verifica se o rótulo do nó existe
        if no_label not in self.mapeamento_indices:
            return [] # Retorna lista vazia se o nó não existe

        idx = self.mapeamento_indices[no_label]
        vizinhos = []
        for i, peso in enumerate(self.adj[idx]):
            if peso != 0:
                vizinhos.append(self.rotulos_nos[i]) # Retorna o rótulo do vizinho, não o índice
        return vizinhos

    def obter_peso_aresta(self, no1_label, no2_label):
        if no1_label not in self.mapeamento_indices or no2_label not in self.mapeamento_indices:
            return 0 # Aresta não existe se um dos nós não existe

        idx1 = self.mapeamento_indices[no1_label]
        idx2 = self.mapeamento_indices[no2_label]
        
        return self.adj[idx1][idx2]
        
    def obter_numero_nos(self):
        return self.num_nos

    def obter_rotulos_nos(self):
        return self.rotulos_nos

In [ ]:
grafo = GrafoMA()

print("Adicionando arestas:")
grafo.adicionar_aresta("A", "B")
grafo.adicionar_aresta("A", 1)
grafo.adicionar_aresta("B", "C", peso=2)
grafo.adicionar_aresta(1, 2, peso=3)
grafo.adicionar_aresta("C", 2)
grafo.adicionar_aresta("D", "A", bidirecional=False)

print("\nGrafo após adicionar arestas:")
print(grafo)

print(f"Rótulos dos nós: {grafo.rotulos_nos}")
print(f"Mapeamento de índices: {grafo.mapeamento_indices}")
print(f"Número de nós: {grafo.obter_numero_nos()}")

In [ ]:
print("\nObtendo vizinhos:")
print(f"Vizinhos de 'A': {grafo.obter_vizinhos('A')}")
print(f"Vizinhos de 'B': {grafo.obter_vizinhos('B')}")
print(f"Vizinhos de 1: {grafo.obter_vizinhos(1)}")
print(f"Vizinhos de 2: {grafo.obter_vizinhos(2)}")
print(f"Vizinhos de 'D': {grafo.obter_vizinhos('D')}")
print(f"Vizinhos de 'X' (não existe): {grafo.obter_vizinhos('X')}")

In [ ]:
print("\nObtendo peso de arestas:")
print(f"Peso da aresta 'A' para 'B': {grafo.obter_peso_aresta('A', 'B')}")
print(f"Peso da aresta 'B' para 'C': {grafo.obter_peso_aresta('B', 'C')}")
print(f"Peso da aresta 1 para 2: {grafo.obter_peso_aresta(1, 2)}")
print(f"Peso da aresta 'A' para 'D': {grafo.obter_peso_aresta('A', 'D')}")
print(f"Peso da aresta 'A' para 'C' (não existe): {grafo.obter_peso_aresta('A', 'C')}")

------------------------------

Implemente o grafo ilustrado pela figura abaixo, utilizando a representação baseada em uma matriz de adjacências. Em seguida, exiba a matriz que representa o grafo.

<img src=https://s3-sa-east-1.amazonaws.com/lcpi/c5b98cc3-2c85-4d9a-b1e8-aaa4b1aba9b3.png width=300, style="background: white">

In [ ]:
g1 = GrafoMA()

g1.adicionar_aresta(1, 3)
g1.adicionar_aresta(3, 2)
g1.adicionar_aresta(3, 4)
g1.adicionar_aresta(2, 5)
g1.adicionar_aresta(4, 5)
g1.adicionar_aresta(4, 6)
g1.adicionar_aresta(5, 5)
g1.adicionar_aresta(5, 6)

g1.adicionar_vertice(0)

print(g1)

Implemente o grafo ponderado ilustrado pela figura abaixo. 

Em seguida, exiba a matriz que representa o grafo.

<img src="https://ucarecdn.com/a67cb888-aa0c-424b-8c7f-847e38dd5691/" width=300>

In [ ]:
g2 = GrafoMA()

g2.adicionar_aresta(0, 1, peso=3)
g2.adicionar_aresta(0, 4, peso=8)
g2.adicionar_aresta(0, 3, peso=7)
g2.adicionar_aresta(1, 3, peso=4)
g2.adicionar_aresta(1, 2, peso=1)
g2.adicionar_aresta(2, 3, peso=2)

print(g2)

------------------------------


#### 2\. Lista de Adjacências (A lista de contatos)

Para cada vértice, temos uma lista de todos os seus vizinhos diretos.

  * **Exemplo prático:**
      * `Porto Alegre`: [`Florianópolis`, `Curitiba`]
      * `São Paulo`: [`Rio de Janeiro`, `Belo Horizonte`, `Curitiba`]
      * `Curitiba`: [`Porto Alegre`, `São Paulo`]

<img src="https://s3-sa-east-1.amazonaws.com/lcpi/9775dad0-c900-42e9-87fa-41d78b73444c.jpg" width=500\>

  * **Quando usar?** É a melhor opção para a maioria dos casos práticos, como redes sociais ou mapas, onde os vértices se conectam apenas a alguns outros. Economiza muita memória.
  * **Desvantagem:** Para verificar se existe uma conexão específica, talvez seja preciso percorrer a lista de vizinhos daquele vértice.

O código abaixo demonstra o exemplo de implementação de um grafo com a Lista de Adjacências

In [ ]:
class GrafoLA:
    def __init__(self):
        # Usaremos um dicionário para mapear rótulos de nós (inteiros ou strings)
        # para suas listas de adjacência.
        self.adj = {}
        # # Para controlar os nós visitados em algoritmos como Busca em Profundidade (DFS) e Busca em Largura (BFS),
        # # usaremos um dicionário que mapeia o rótulo do nó para um status (0 para não visitado, 1 para visitado).
        # self.__visitado = {}
        # self.ordem_visitados = []

    def __repr__(self):
        return self.__str__()
    
    def __str__(self):
        # Retorna uma representação legível do grafo.
        # Ex: "A: ['B', 'C']\nB: ['A']"
        return '\n'.join([f"{no}: {vizinhos}" for no, vizinhos in self.adj.items()])

    def adicionar_vertice(self, n1):
        if n1 not in self.adj:
            self.adj[n1] = []

    def adicionar_aresta(self, n1, n2, peso=1, bidirecional=True):
        # Garante que os nós existam no dicionário de adjacência
        if n1 not in self.adj:
            self.adj[n1] = []
        if n2 not in self.adj:
            self.adj[n2] = []

        # Adiciona a aresta de n1 para n2
        self.adj[n1].append((n2, peso))

        # Se for bidirecional, adiciona a aresta de n2 para n1
        if bidirecional:
            self.adj[n2].append((n1, peso))

    def obter_vizinhos(self, n):
        # Retorna a lista de vizinhos de um nó.
        # Retorna uma lista vazia se o nó não existir.
        return self.adj.get(n, [])

In [ ]:
print("--- Grafo com Inteiros ---")
grafo_numeros = GrafoLA()
grafo_numeros.adicionar_aresta(0, 1, peso=3)
grafo_numeros.adicionar_aresta(0, 2, bidirecional=False)
grafo_numeros.adicionar_aresta(1, 3)
print(grafo_numeros) # Saída: 0: [1, 2]\n1: [0, 3]\n2: [0]\n3: [1]

print("\nVizinhos de 0:", grafo_numeros.obter_vizinhos(0)) # Saída: Vizinhos de 0: [1, 2]
print("Vizinhos de 5 (não existe):", grafo_numeros.obter_vizinhos(5)) # Saída: Vizinhos de 5 (não existe): []

In [ ]:
print("--- Grafo com Strings ---")
grafo_nomes = GrafoLA()
grafo_nomes.adicionar_aresta("Alice", "Bob")
grafo_nomes.adicionar_aresta("Bob", "Carol", bidirecional=False) # Apenas Bob -> Carol
grafo_nomes.adicionar_aresta("Carol", "David")
print(grafo_nomes) # Saída: Alice: ['Bob']\nBob: ['Alice', 'Carol']\nCarol: ['Bob', 'David']\nDavid: ['Carol']

print("\nVizinhos de 'Alice':", grafo_nomes.obter_vizinhos("Alice")) # Saída: Vizinhos de 'Alice': ['Bob']
print("Vizinhos de 'Bob':", grafo_nomes.obter_vizinhos("Bob"))     # Saída: Vizinhos de 'Bob': ['Alice', 'Carol']

In [ ]:

print("--- Grafo com Strings e Inteiros Misturados ---")
grafo_misto = GrafoLA()
grafo_misto.adicionar_aresta("Servidor1", 10)
grafo_misto.adicionar_aresta(10, "Servidor2")
grafo_misto.adicionar_aresta("Servidor2", "Impressora3")
print(grafo_misto)

print("\nVizinhos de 'Servidor1':", grafo_misto.obter_vizinhos("Servidor1"))
print("Vizinhos de 10:", grafo_misto.obter_vizinhos(10))

------------------------------

Implemente o grafo ilustrado pela figura abaixo, utilizando a representação baseada em uma lista de adjacências. Em seguida, observe a lista de adjacências que representa o grafo.

<img src=https://s3-sa-east-1.amazonaws.com/lcpi/c5b98cc3-2c85-4d9a-b1e8-aaa4b1aba9b3.png width=300, style="background: white">

In [5]:
g = GrafoLA()

g.adicionar_vertice(0)
g.adicionar_aresta(1, 3)

print(g)

0: []
1: [(3, 1)]
3: [(1, 1)]


Por que incluir um vértice sozinho?

Pensem em um jogo de rpg, onde temos um mapa para explorar. Em um determinado momento, conversamos com um NPC e ele passa uma missão para ser feita na cidade 0, mas ainda não sabemos o caminho até zero.

Mas sabemos que ela está lá.

Aqui é a mesma coisa. Não sabemos qual a ligação do vértice 0 com os demais vértices do grafo, mas sabemos que o vértice 0 existe na nossa representação.

Fazendo um novo paralelo, imaginem que o grafo acima mostra as ligações entre computadores em uma rede local.

O computador 0 não está conectado com ninguém porque está desligado. Mas quando for ligado, o grafo vai se atualizar e conectar o 0 com outro computador.

------------------------------

Pergunta: como podemos representar grafos ponderados com lista de adjacência?

Com uma tupla

## Percurso em Grafo: Como visitar todo mundo?

**percurso** ou **busca**: passar por todos os vértices do grafo de forma organizada.

<img src="https://miro.medium.com/max/1280/0*miG6xdyYzdvrB67S.gif" width=450\>

&nbsp;

### 1\. Busca em profundidade (DFS - *Depth-First Search*)

A DFS é a estratégia do "explorador de labirintos".

  - **Como funciona?** Ela escolhe um caminho e vai o mais fundo possível, até chegar a um beco sem saída. Só então ela volta (faz o *backtracking*) e tenta o próximo caminho disponível a partir da última encruzilhada.
  - **Exemplo do mundo real**
  
    **Detectando ciclos em redes (Como circuitos elétricos ou outros)**: Estamos projetando um circuito elétrico ou analisando uma rede de tubulações. É crucial saber se existe um ciclo (um "loop" onde o fluxo pode voltar ao ponto de partida). Um ciclo pode indicar um problema de design, um curto-circuito ou até mesmo um "loop infinito" em um programa de computador. A DFS é ótima para isso porque, ao explorar profundamente um caminho, se ela encontrar um nó que já foi visitado e ainda está sendo processado na mesma "ramificação" da busca, ela detecta um ciclo!
  - **Exemplo prático:** Você está em um labirinto (o nosso "grafo") e quer encontrar algo ou simplesmente explorar todos os caminhos. A Busca em Profundidade (DFS) é como se você decidisse **ir o mais fundo possível por um caminho antes de voltar e tentar outro.**]

  Você começa em um ponto (o **nó inicial**). Dali, você escolhe um caminho e o segue até o fim, sem desviar. Só quando não há mais para onde ir por aquele caminho, você volta um passo e tenta um caminho diferente, que ainda não explorou.

&nbsp;

Abaixo temos um exemplo de como a busca em profundidade percorreria esse grafo partindo do nó 0.

<img src="https://s3-sa-east-1.amazonaws.com/lcpi/0084b080-e980-46d0-8694-55895333a3e7.jpg" width=600>

Abaixo podemos visualizar a implementação de uma busca em profundidade (DFS) em um grafo como lista de adjacências.

In [ ]:
class GrafoLA:
    def __init__(self):
        # Usaremos um dicionário para mapear rótulos de nós (inteiros ou strings)
        # para suas listas de adjacência.
        self.adj = {}
        # Para controlar os nós visitados em algoritmos como Busca em Profundidade (DFS) e Busca em Largura (BFS),
        # usaremos um dicionário que mapeia o rótulo do nó para um status (0 para não visitado, 1 para visitado).
        self.__visitado = {}
        self.ordem_visitados = []

    def __str__(self):
        # Retorna uma representação legível do grafo.
        # Ex: "A: ['B', 'C']\nB: ['A']"
        return '\n'.join([f"{no}: {vizinhos}" for no, vizinhos in self.adj.items()])

    def adicionar_vertice(self, n1):
        if n1 not in self.adj:
            self.adj[n1] = []
            
    def adicionar_aresta(self, n1, n2, bidirecional=True):
        # Garante que os nós existam no dicionário de adjacência
        if n1 not in self.adj:
            self.adj[n1] = []
        if n2 not in self.adj:
            self.adj[n2] = []

        # Adiciona a aresta de n1 para n2
        self.adj[n1].append(n2)

        # Se for bidirecional, adiciona a aresta de n2 para n1
        if bidirecional:
            self.adj[n2].append(n1)

    def obter_vizinhos(self, n):
        # Retorna a lista de vizinhos de um nó.
        # Retorna uma lista vazia se o nó não existir.
        return self.adj.get(n, [])
    
    # Comum para DFS e BFS
    def __visitar(self, no): # 'node' -> 'no'
        # Marca um nó como visitado e o adiciona à ordem de visita.
        self.__visitado[no] = 1
        self.ordem_visitados.append(no)

    # DFS - Busca em Profundidade
    def __dfs(self, n):
        # Implementação recursiva da Busca em Profundidade (DFS).
        self.__visitar(n)
        for vizinho in self.obter_vizinhos(n):
            # Verifica se o vizinho não foi visitado.
            if self.__visitado.get(vizinho, 0) == 0:
                self.__dfs(vizinho)

    def DFS(self, no_inicial):
        # Inicia a Busca em Profundidade (DFS) a partir de um nó.
        # Redefine o status de visitado para todos os nós para cada nova execução de DFS.
        self.__visitado = {no: 0 for no in self.adj}
        self.ordem_visitados = []
        if no_inicial in self.adj:
            self.__dfs(no_inicial)
        return self.ordem_visitados
    # DFS - Busca em profundidade

In [ ]:
# Pseudocodigo da DFS

# FUNÇÃO __visitar(self, no):
#   Marca um nó como visitado e o adiciona à ordem de visita.
#   DEFINIR self.__visitado[no] = 1
#   ADICIONAR no A self.ordem_visitados
# FIM FUNÇÃO

# FUNÇÃO __dfs(self, n):
#   Implementação recursiva da Busca em Profundidade (DFS).
#   CHAMAR __visitar(self, n)
#   PARA CADA vizinho EM self.obter_vizinhos(n):
#     # Verifica se o vizinho não foi visitado.
#     SE self.__visitado.GET(vizinho, 0) É IGUAL A 0:
#       CHAMAR self.__dfs(vizinho)
#   FIM PARA
# FIM FUNÇÃO

# FUNÇÃO DFS(self, no_inicial):
#   Inicia a Busca em Profundidade (DFS) a partir de um nó.
#   Redefine o status de visitado para todos os nós para cada nova execução de DFS.
#   DEFINIR self.__visitado = UM DICIONÁRIO ONDE CADA no EM self.adj TEM VALOR 0
#   DEFINIR self.ordem_visitados = UMA LISTA VAZIA
#   SE no_inicial ESTÁ EM self.adj:
#     CHAMAR self.__dfs(no_inicial)
#   FIM SE
#   RETORNAR self.ordem_visitados
# FIM FUNÇÃO

#### As Ferramentas da DFS: O que ela precisa?

Para a DFS funcionar, ela usa algumas "ferramentas" ou informações:

- `self.adj` **(Suas adjacências/vizinhos)**: Pense nisso como o mapa do labirinto. Para cada canto (nó), ele te diz quais outros cantos (vizinhos) você pode ir diretamente.

- `self.__visitado` **(Seu diário de bordo)**: Este é um diário onde você anota por quais cantos (nós) você já passou. Isso é super importante para não ficar andando em círculos e para saber quais caminhos você já explorou. Se um nó está no diário como "visitado", você não precisa ir lá de novo.

- `self.ordem_visitados` **(Seu caminho percorrido)**: Uma lista que guarda a sequência de cantos (nós) que você visitou na sua exploração.

#### Entendendo as Funções: Passo a Passo

1. `__visitar(self, no)`: **Marcando o caminho**

    Esta é uma função simples, mas fundamental. Toda vez que você chega a um novo canto (nó) no labirinto:

- Você o marca no seu diário (`self.__visitado[no] = 1`) para dizer "já estive aqui".

- Você anota esse canto na sua lista de "caminhos percorridos" (`self.ordem_visitados.append(no)`).

&nbsp;


2. `__dfs(self, n)`: **A exploração "profunda" (A alma da DFS)**

    Esta é a função principal que faz a exploração acontecer, e ela é **recursiva**. Isso significa que ela se chama várias vezes para ir mais fundo nos caminhos.

- `self.__visitar(n)`: Primeiro, assim que você chega a um nó `n`, você o marca como visitado.

- `for vizinho in self.obter_vizinhos(n):`: Agora, você olha para todos os caminhos que saem desse nó `n` (seus vizinhos).

- `if self.__visitado.get(vizinho, 0) == 0:`: Para cada vizinho, você consulta seu diário de bordo. Se o vizinho ainda não foi visitado (o valor é `0` ou ele nem está no diário, o que significa `0`), então:

- `self.__dfs(vizinho)`: Você decide "ir fundo" por esse vizinho. A função `__dfs` é chamada novamente, mas agora para o `vizinho`. É como se você entrasse em uma nova parte do labirinto, e a função continua a partir dali, indo o mais fundo possível.

- **Volta**: Só quando a `__dfs` para um vizinho termina (ou seja, não há mais caminhos novos a partir daquele vizinho e seus descendentes), ela "volta" para onde estava antes e tenta o próximo vizinho (se houver).

3. `DFS(self, no_inicial)`: **Onde a aventura começa!**

    Esta é a função que você chama para iniciar a busca:

- `self.__visitado = {no: 0 for no in self.adj}`: Antes de começar uma nova exploração, você limpa seu diário de bordo! Marca todos os nós do labirinto como "não visitados" (`0`), para que você possa começar do zero.

- `self.ordem_visitados = []`: Zera sua lista de caminhos percorridos também.

- `if no_inicial in self.adj:`: Verifica se o nó de onde você quer começar a exploração (`no_inicial`) realmente existe no seu mapa (`self.adj`). Se não existir, não há como começar, então a função não faz nada.

- `self.__dfs(no_inicial)`: Agora sim, a magia começa! Você chama a função `__dfs` para iniciar a exploração a partir do `no_inicial`.

- `return self.ordem_visitados`: Depois que a `__dfs` termina toda a exploração (ou seja, ela visitou todos os cantos alcançáveis a partir do `no_inicial`), a função `DFS` te devolve a lista completa da ordem em que os nós foram visitados.

---------------------------

<img src="https://s3-sa-east-1.amazonaws.com/lcpi/9775dad0-c900-42e9-87fa-41d78b73444c.jpg" width=500\>

In [32]:
g3 = GrafoLA()
g3.adicionar_aresta(0, 1)
g3.adicionar_aresta(0, 2)
g3.adicionar_aresta(1, 2)
g3.adicionar_aresta(1, 3)
g3.adicionar_aresta(2, 4)
g3.adicionar_aresta(3, 4)
g3.adicionar_aresta(4, 5)
g3.adicionar_aresta(5, 6)
print(g3, "\n")

ordem_visitados = g3.DFS(0)
ordem_visitados

0: [1, 2]
1: [0, 2, 3]
2: [0, 1, 4]
3: [1, 4]
4: [2, 3, 5]
5: [4, 6]
6: [5] 



[0, 1, 2, 4, 3, 5, 6]


### 2\. Busca em Largura (BFS - *Breadth-First Search*)

A BFS é a estratégia do "efeito cascata" ou das "camadas".

  - **Como funciona?** A partir de um ponto inicial, ela primeiro visita todos os vizinhos diretos (a 1ª camada). Depois, visita todos os vizinhos *desses vizinhos* (a 2ª camada), e assim por diante, se espalhando como as ondas na água.
  - **Exemplo do mundo real**: Encontrar o caminho mais curto em um sistema de navegação, ou amigos em redes sociais.
  - **Exemplo prático:** Em uma rede social, como o LinkedIn, para encontrar os seus "amigos de amigos". A BFS primeiro lista todos os seus amigos diretos (1ª camada) e depois lista todos os amigos deles (2ª camada).


&nbsp;

A figura abaixo mostra um exemplo gráfico desse comportamento em camadas.

<img src="https://s3-sa-east-1.amazonaws.com/lcpi/a67dd98f-8578-4bda-a710-903d7de9ceb6.jpg" width=600>

&nbsp;

Notem que a ordem de visitação imposta pela BFS é parcial.

Existem diferentes ordens possíveis que podem ser feitas pelo percurso BFS, o importante é respeitar as camadas.

Por exemplo, não importa se 1 foi visitado antes do 3 ou se o 3 foi visitado antes do 1 desde que ambos tenham sido visitados antes de 2, 4 e 5.

In [2]:
class GrafoLA:
    def __init__(self):
        # Usaremos um dicionário para mapear rótulos de nós (inteiros ou strings)
        # para suas listas de adjacência.
        self.adj = {}
        # Para controlar os nós visitados em algoritmos como Busca em Profundidade (DFS) e Busca em Largura (BFS),
        # usaremos um dicionário que mapeia o rótulo do nó para um status (0 para não visitado, 1 para visitado).
        self.__visitado = {}
        self.ordem_visitados = []

    def __str__(self):
        # Retorna uma representação legível do grafo.
        # Ex: "A: ['B', 'C']\nB: ['A']"
        return '\n'.join([f"{no}: {vizinhos}" for no, vizinhos in self.adj.items()])

    def adicionar_vertice(self, n1):
        if n1 not in self.adj:
            self.adj[n1] = []
            
    def adicionar_aresta(self, n1, n2, bidirecional=True):
        # Garante que os nós existam no dicionário de adjacência
        if n1 not in self.adj:
            self.adj[n1] = []
        if n2 not in self.adj:
            self.adj[n2] = []

        # Adiciona a aresta de n1 para n2
        self.adj[n1].append(n2)

        # Se for bidirecional, adiciona a aresta de n2 para n1
        if bidirecional:
            self.adj[n2].append(n1)

    def obter_vizinhos(self, n):
        # Retorna a lista de vizinhos de um nó.
        # Retorna uma lista vazia se o nó não existir.
        return self.adj.get(n, [])

    # Comum para DFS e BFS
    def __visitar(self, no): # 'node' -> 'no'
        # Marca um nó como visitado e o adiciona à ordem de visita.
        self.__visitado[no] = 1
        self.ordem_visitados.append(no)

    # DFS - Busca em Profundidade
    def __dfs(self, n):
        # Implementação recursiva da Busca em Profundidade (DFS).
        self.__visitar(n)
        for vizinho in self.obter_vizinhos(n):
            # Verifica se o vizinho não foi visitado.
            if self.__visitado.get(vizinho, 0) == 0:
                self.__dfs(vizinho)

    def DFS(self, no_inicial):
        # Inicia a Busca em Profundidade (DFS) a partir de um nó.
        # Redefine o status de visitado para todos os nós para cada nova execução de DFS.
        self.__visitado = {no: 0 for no in self.adj}
        self.ordem_visitados = []
        if no_inicial in self.adj:
            self.__dfs(no_inicial)
        return self.ordem_visitados
    # DFS - Busca em profundidade

    # BFS - Busca em largura
    def BFS(self, no_inicial):
        # Inicia a Busca em Largura (BFS) a partir de um nó.
        # Redefine o status de visitado para todos os nós.
        self.__visitado = {no: 0 for no in self.adj}
        self.ordem_visitados = []
        fila = []  # Usaremos uma lista como fila para a BFS

        if no_inicial not in self.adj:
            return self.ordem_visitados

        self.__visitar(no_inicial)
        fila.append(no_inicial)

        while fila:
            no_atual = fila.pop(0)  # Pega o primeiro nó da fila
            for vizinho in self.obter_vizinhos(no_atual):
                if self.__visitado.get(vizinho, 0) == 0:
                    self.__visitar(vizinho)
                    fila.append(vizinho)
        return self.ordem_visitados
    # BFS - Busca em largura

In [ ]:
# FUNÇÃO BFS(self, no_inicial):
#   Inicia a Busca em Largura (BFS) a partir de um nó.
#   Redefine o status de visitado para todos os nós.

#   DEFINIR self.__visitado = UM DICIONÁRIO ONDE CADA no EM self.adj TEM VALOR 0
#   DEFINIR self.ordem_visitados = UMA LISTA VAZIA
#   DEFINIR fila = UMA LISTA VAZIA // Usaremos uma lista como fila para a BFS

#   SE no_inicial NÃO ESTÁ EM self.adj:
#     RETORNAR self.ordem_visitados // O nó inicial não existe no grafo
#   FIM SE

#   CHAMAR self.__visitar(no_inicial) // Marca o nó inicial como visitado
#   ADICIONAR no_inicial À fila

#   ENQUANTO fila NÃO ESTÁ VAZIA:
#     DEFINIR no_atual = REMOVER PRIMEIRO ELEMENTO DA fila // Pega o primeiro nó da fila
#     PARA CADA vizinho EM self.obter_vizinhos(no_atual):
#       SE self.__visitado.GET(vizinho, 0) É IGUAL A 0: // Verifica se o vizinho não foi visitado.
#         CHAMAR self.__visitar(vizinho) // Marca o vizinho como visitado
#         ADICIONAR vizinho À fila
#       FIM SE
#     FIM PARA
#   FIM ENQUANTO

#   RETORNAR self.ordem_visitados
# FIM FUNÇÃO

Pensando na ideia de percorrer o labirinto, a estratégia do BFS seria um pouco diferente.

Se a Busca em Profundidade (DFS) era como ir o mais fundo possível em um caminho, a Busca em Largura (BFS) é como explorar o labirinto **passo a passo, visitando todos os cantos que estão mais próximos antes de ir para os mais distantes.**

Imagine que você joga uma gota de tinta no seu ponto de partida. Essa gota se espalha para todos os cantos diretamente conectados a ela. Depois, de todos esses novos cantos, ela se espalha para os próximos, e assim por diante. A BFS faz exatamente isso: ela visita todos os "vizinhos" de um ponto, depois todos os "vizinhos dos vizinhos", e assim por diante, em uma ordem crescente de distância.

#### As Ferramentas da BFS: O que ela precisa?

A BFS usa as mesmas "ferramentas" básicas da DFS, mas com uma adição crucial:

- `self.adj` **(Suas adjacências/vizinhos)**: O mapa do labirinto, mostrando quais cantos (nós) estão conectados diretamente.

- `self.__visitado` **(Seu diário de bordo)**: O diário onde você anota por quais cantos (nós) você já passou, para não revisitar.

- `self.ordem_visitados` **(Seu caminho percorrido)**: Uma lista que guarda a sequência de cantos (nós) que você visitou.

- `fila` **(Sua Lista de Espera)**: Esta é a ferramenta mais importante da BFS. É uma lista onde você coloca os cantos que você **descobriu, mas que ainda não explorou**. Quando você termina de explorar um canto, você pega o próximo da fila.

#### Entendendo a função `BFS(self, no_inicial)`: Passo a passo

Esta é a função que orquestra toda a exploração em largura:

1. **Preparação**:

- `self.__visitado = {no: 0 for no in self.adj}`: Antes de começar, você limpa seu diário de bordo. Todos os cantos do labirinto são marcados como "não visitados" (`0`).

- `self.ordem_visitados = []`: Zera sua lista de caminhos percorridos.

- `fila = []`: Você cria uma **fila vazia**. Pense nela como uma fila de pessoas esperando para entrar em uma atração: quem chega primeiro é o primeiro a ser atendido.

&nbsp;

2. **Início da Aventura**:

- `if no_inicial not in self.adj:`: Primeiro, verifica se o canto (`no_inicial`) de onde você quer começar existe no seu mapa. Se não existir, a busca não tem como iniciar, e a função retorna a lista vazia.

- `self.__visitar(no_inicial)`: Se o nó existe, você o marca no diário como visitado. Ele é o primeiro canto que você explora.

- `fila.append(no_inicial)`: E, como ele é o primeiro a ser explorado, você o coloca na sua fila de espera.

&nbsp;

3. **Exploração em Largura (O Loop Principal)**:

- `while fila:`: Este é o coração da BFS. O processo continua enquanto houver cantos na sua fila esperando para serem explorados.

- `no_atual = fila.pop(0)`: Aqui está a chave da BFS! Você pega o **primeiro** canto que entrou na fila (por isso é "em largura", você atende os mais antigos/próximos primeiro).

    Esse é o `no_atual` que você vai explorar agora.

- `for vizinho in self.obter_vizinhos(no_atual):`: Você olha para todos os caminhos que saem do seu `no_atual` (todos os seus vizinhos diretos).

- `if self.__visitado.get(vizinho, 0) == 0:`: Para cada vizinho, você consulta seu diário. Se o vizinho ainda não foi visitado (`0`):

    - `self.__visitar(vizinho)`: Você o marca como visitado.

    - `fila.append(vizinho)`: E **adiciona esse vizinho ao final da fila**. Ele vai esperar a vez dele para ser explorado.

&nbsp;

4. **Fim da Busca**:

- `return self.ordem_visitados`: Quando a fila estiver vazia (significa que você explorou todos os cantos alcançáveis), a função te devolve a ordem em que você visitou cada um deles.

---------------------------

<img src="https://s3-sa-east-1.amazonaws.com/lcpi/9775dad0-c900-42e9-87fa-41d78b73444c.jpg" width=500\>

In [4]:
g3 = GrafoLA()
g3.adicionar_aresta(0, 1)
g3.adicionar_aresta(0, 2)
g3.adicionar_aresta(1, 2)
g3.adicionar_aresta(1, 3)
g3.adicionar_aresta(2, 4)
g3.adicionar_aresta(3, 4)
g3.adicionar_aresta(4, 5)
g3.adicionar_aresta(5, 6)
print(g3, "\n")

ordem_visitados = g3.BFS(4)
ordem_visitados

0: [1]
1: [0]
0: [1, 2]
1: [0]
2: [0]
0: [1, 2]
1: [0, 2]
2: [0, 1]
0: [1, 2]
1: [0, 2, 3]
2: [0, 1, 4]
3: [1, 4]
4: [2, 3, 5]
5: [4, 6]
6: [5] 



[4, 2, 3, 5, 0, 1, 6]